# Q1. Install MLflow

In [1]:
# Q1
! mlflow --version

mlflow, version 2.13.0


# Q2. Download and preprocess the data

In [2]:
links = [
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet",
    "https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-03.parquet"
]

# download data to data subdirectory replace if exists
for link in links:
    ! wget -N -P data $link

--2024-05-24 10:58:54--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-01.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.161.3.141, 18.161.3.209, 18.161.3.119, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.161.3.141|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘data/green_tripdata_2023-01.parquet’ not modified on server. Omitting download.

--2024-05-24 10:58:55--  https://d37ci6vzurychx.cloudfront.net/trip-data/green_tripdata_2023-02.parquet
Resolving d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)... 18.161.3.119, 18.161.3.141, 18.161.3.209, ...
Connecting to d37ci6vzurychx.cloudfront.net (d37ci6vzurychx.cloudfront.net)|18.161.3.119|:443... connected.
HTTP request sent, awaiting response... 304 Not Modified
File ‘data/green_tripdata_2023-02.parquet’ not modified on server. Omitting download.

--2024-05-24 10:58:56--  https://d37ci6vzurych

In [3]:
# download preprocess_data.py
! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py

--2024-05-24 10:59:06--  https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/preprocess_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.111.133, 185.199.108.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.111.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2510 (2.5K) [text/plain]
Saving to: ‘preprocess_data.py’

preprocess_data.py  100%[===================>]   2.45K  --.-KB/s    in 0s      

Last-modified header missing -- time-stamps turned off.
2024-05-24 10:59:06 (38.7 MB/s) - ‘preprocess_data.py’ saved [2510/2510]



In [4]:
! python preprocess_data.py --raw_data_path TAXI_DATA_FOLDER --dest_path ./output

Traceback (most recent call last):
  File "/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/preprocess_data.py", line 83, in <module>
    run_data_prep()
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/click/core.py", line 1157, in __call__
    return self.main(*args, **kwargs)
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/click/core.py", line 1078, in main
    rv = self.invoke(ctx)
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/click/core.py", line 1434, in invoke
    return ctx.invoke(self.callback, **ctx.params)
  File "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/click/core.py", line 783, in invoke
    return __callback(*args, **kwargs)
  File "/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/preprocess_data.py", line 50, in run_data_prep
    df_train = read_dataframe(
  File "/workspaces/mlops-zoomcamp/02-experiment-tracking/homework/preprocess_data.py", line 15, in read_dataframe
    df = pd.read_parquet(filename)


In [5]:
# Q2
# number of files in output directory
import os
output_dir = './output'
num_files = len([name for name in os.listdir(output_dir) if os.path.isfile(os.path.join(output_dir, name))])
num_files

4

# Q3. Train a model with autolog

In [6]:
# download train.py
# ! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/train.py

In [20]:
# run modified train.py
! python train.py

2024/05/24 11:22:13 INFO mlflow.tracking.fluent: Experiment with name 'train' does not exist. Creating a new experiment.
2024/05/24 11:22:13 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2024/05/24 11:22:13 WARNING mlflow.sklearn: Failed to log training dataset information to MLflow Tracking. Reason: 'numpy.ndarray' object has no attribute 'toarray'
2024/05/24 11:22:27 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/opt/conda/envs/hw2-env/lib/python3.9/site-packages/_distutils_hack/__init__.py:26: UserWarning: Setuptools is replacing distutils."


In [21]:
# Q3
# get min_samples_split from last run
import mlflow
import pandas as pd

# Get last run
experiments = mlflow.search_runs()
last_run = experiments[experiments['status'] == 'FINISHED'].sort_values(by='end_time', ascending=False).head(1)

# Get run_id
run_id = last_run['run_id'].values[0]

# Get run
run = mlflow.get_run(run_id)

# Get run data
data = run.data

# Get params
params = data.params

# Get min_samples_split
min_samples_split = params['min_samples_split']
min_samples_split

'2'

# Q4. Launch the tracking server locally

Launch mlflow locally select a `hw2.sqlite` db for the backend store and
a folder called `artifacts` for the artifacts store
```bash
mlflow ui --backend-store-uri sqlite:///hw2.sqlite --artifacts-destination ./artifacts
```
References:
https://stackoverflow.com/questions/75057477/mlflow-server-difference-between-default-artifact-root-and-artifacts
https://mlflow.org/docs/latest/cli.html#mlflow-artifacts

In [16]:
# ! mlflow ui --backend-store-uri sqlite:///hw2.sqlite --artifacts-destination ./artifacts

In [17]:
# Q4 answer: artifacts-destination

# Q5. Tune model hyperparameters

In [18]:
# download hpo.py
# ! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/hpo.py

In [25]:
! python hpo.py

100%|██████████| 15/15 [01:00<00:00,  4.03s/trial, best loss: 5.335419588556921]


# Q6. Promote the best model to the model registry

In [27]:
# download register_model.py
# ! wget -N https://raw.githubusercontent.com/DataTalksClub/mlops-zoomcamp/main/cohorts/2024/02-experiment-tracking/homework/register_model.py

In [29]:
# Q6
! python register_model.py

Model ID: eed75945e0f34c33917f88ec59299907 | Test RMSE: 5.567
Successfully registered model 'best_run_model'.
2024/05/24 12:04:47 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: best_run_model, version 1
Created version '1' of model 'best_run_model'.
